In [22]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time
import json

In [23]:
# Application Config
EnableColorStream = 0
ColorStreamResWidth = 1920
ColorStreamResHeight = 1080
ColorStreamFPS = 30

EnableDepthStream = 1
DepthStreamResWidth = 1280
DepthStreamResHeight = 720
DepthStreamFPS = 30

capture = 0

In [24]:
DS5_product_ids = ["0AD1", "0AD2", "0AD3", "0AD4", "0AD5", "0AF6", "0AFE", "0AFF", "0B00", "0B01", "0B03", "0B07"]

def find_device_that_supports_advanced_mode() :
    ctx = rs.context()
    ds5_dev = rs.device()
    devices = ctx.query_devices();
    for dev in devices:
        if dev.supports(rs.camera_info.product_id) and str(dev.get_info(rs.camera_info.product_id)) in DS5_product_ids:
            if dev.supports(rs.camera_info.name):
                print("Found device that supports advanced mode:", dev.get_info(rs.camera_info.name))
            return dev
    raise Exception("No device that supports advanced mode was found")

In [25]:
dev = find_device_that_supports_advanced_mode()
advnc_mode = rs.rs400_advanced_mode(dev)

current_std_depth_control_group = advnc_mode.get_depth_control()
current_std_depth_control_group.deepSeaMedianThreshold = 789
current_std_depth_control_group.deepSeaNeighborThreshold = 12
current_std_depth_control_group.deepSeaSecondPeakThreshold = 600
current_std_depth_control_group.lrAgreeThreshold = 19
current_std_depth_control_group.minusDecrement = 23
current_std_depth_control_group.plusIncrement = 3
current_std_depth_control_group.scoreThreshA = 54
current_std_depth_control_group.scoreThreshB = 887
current_std_depth_control_group.textureCountThreshold = 0
current_std_depth_control_group.textureDifferenceThreshold = 0
advnc_mode.set_depth_control(current_std_depth_control_group)

current_std_rsm_group = advnc_mode.get_rsm()
current_std_rsm_group.diffThresh = 1.816750049591
current_std_rsm_group.removeThresh = 81
current_std_rsm_group.rsmBypass = 0
current_std_rsm_group.sloRauDiffThresh = 1
advnc_mode.set_rsm(current_std_rsm_group)

current_std_rsvc_group = advnc_mode.get_rau_support_vector_control()
current_std_rsvc_group.minEast = 3
current_std_rsvc_group.minNorth = 1
current_std_rsvc_group.minNSsum = 6
current_std_rsvc_group.minSouth = 3
current_std_rsvc_group.minWest = 5
current_std_rsvc_group.minWEsum = 11
current_std_rsvc_group.uShrink = 3
current_std_rsvc_group.vShrink = 1
advnc_mode.set_rau_support_vector_control(current_std_rsvc_group)

curret_std_color_control_group = advnc_mode.get_color_control()
curret_std_color_control_group.disableRAUColor = 0
curret_std_color_control_group.disableSADColor = 0
curret_std_color_control_group.disableSADNormalize = 0
curret_std_color_control_group.disableSLOLeftColor = 0
curret_std_color_control_group.disableSLORightColor = 1
advnc_mode.set_color_control(curret_std_color_control_group)

current_std_rctc_group = advnc_mode.get_rau_thresholds_control()
current_std_rctc_group.rauDiffThresholdBlue = 14;
current_std_rctc_group.rauDiffThresholdGreen = 723;
current_std_rctc_group.rauDiffThresholdRed = 186;
advnc_mode.set_rau_thresholds_control(current_std_rctc_group)

current_std_sctc_group = advnc_mode.get_slo_color_thresholds_control()
current_std_sctc_group.diffThresholdBlue = 16
current_std_sctc_group.diffThresholdGreen = 259
current_std_sctc_group.diffThresholdRed = 896
advnc_mode.set_slo_color_thresholds_control(current_std_sctc_group)

current_std_spc_group = advnc_mode.get_slo_penalty_control()
current_std_spc_group.sloK1Penalty = 132
current_std_spc_group.sloK1PenaltyMod1 = 77
current_std_spc_group.sloK1PenaltyMod2 = 234
current_std_spc_group.sloK2Penalty = 342
current_std_spc_group.sloK2PenaltyMod1 = 390
current_std_spc_group.sloK2PenaltyMod2 = 151
advnc_mode.set_slo_penalty_control(current_std_spc_group)

current_std_hdad_group = advnc_mode.get_hdad()
current_std_hdad_group.lambdaAD = 935
current_std_hdad_group.lambdaCensus = 26
advnc_mode.set_hdad(current_std_hdad_group)

serialized_string = advnc_mode.serialize_json()
as_json_object = json.loads(serialized_string)
json_string = str(as_json_object).replace("'", '\"')
advnc_mode.load_json(json_string)


Found device that supports advanced mode: Intel RealSense 410


'\n\n\np.spc.sloK1Penalty = 132;\np.spc.sloK1PenaltyMod1 = 77;\np.spc.sloK1PenaltyMod2 = 234;\np.spc.sloK2Penalty = 342;\np.spc.sloK2PenaltyMod1 = 390;\np.spc.sloK2PenaltyMod2 = 151;\np.hdad.lambdaAD = 935;\np.hdad.lambdaCensus = 26;\n\n\nserialized_string = advnc_mode.serialize_json()\nas_json_object = json.loads(serialized_string)\njson_string = str(as_json_object).replace("\'", \'"\')\nadvnc_mode.load_json(json_string)\n'

In [26]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColorStream:
    config.enable_stream(
        rs.stream.color, ColorStreamResWidth, ColorStreamResHeight, rs.format.bgr8, ColorStreamFPS)

if EnableDepthStream:
    config.enable_stream(
        rs.stream.depth, DepthStreamResWidth, DepthStreamResHeight, rs.format.z16, DepthStreamFPS)

# Start streaming
pipeline.start(config)

In [27]:
while True:
    # Wait for a coherent pair of frames: depth and color
    frames = pipeline.wait_for_frames()
    
    if EnableColorStream:
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue
    
    if EnableDepthStream:
        depth_frame = frames.get_depth_frame()
        if not depth_frame:
            continue

    # Convert images to numpy arrays
    if EnableColorStream:
        color_image = np.asanyarray(color_frame.get_data())
    
    if EnableDepthStream:
        depth_image = np.asanyarray(depth_frame.get_data())
    
    depth_image = cv2.convertScaleAbs(depth_image, None, 0.1, 0)
    #depth_image = cv2.equalizeHist(depth_image)    
    depth_colormap = cv2.applyColorMap(depth_image, cv2.COLORMAP_JET)

    # Show images
    if EnableColorStream:
        cv2.imshow('Color image', color_image)
    
    if EnableDepthStream:
        cv2.imshow('Depth image', depth_colormap)
    
    # Screen capture
    if capture:
        if EnableColorStream:
            fileName = "../imageCapture/color_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, color_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        
        if EnableDepthStream:
            fileName = "../imageCapture/depth_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, depth_colormap, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        
        capture = 0
    
    # Keyboard command
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('c') or getKey is ord('C'):
        capture = 1
    elif getKey is ord('q') or getKey is ord('Q'):
        break

In [28]:
# Stop streaming
cv2.destroyAllWindows()
pipeline.stop()